In [1]:
from dotenv import load_dotenv

In [61]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders.base import BaseLoader
from langchain.schema import Document
import json
import re
from langchain.text_splitter import TextSplitter
import numpy as np
from typing import List
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
import re
# from langchain.text_splitter import SemanticChunker
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pickle
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings


In [47]:
openai_api_key = os.getenv("OPENAI_API_KEY")

# 1. 문서 로드(Load Documents)

In [26]:
class JSONLLoader(BaseLoader):
    def __init__(self, file_path: str):
        self.file_path = file_path

    def load(self):
        documents = []
        seq_num = 1
        
        with open(self.file_path, 'r', encoding='utf-8') as file:
            for line in file:
                data = json.loads(line)
                doc = Document(
                    page_content=data['content'],
                    metadata={
                        'docid': data['docid'],
                        'src': data.get('src', ''),  # 'src' 필드가 없을 경우 빈 문자열 사용
                        'source': self.file_path,
                        'seq_num': seq_num,
                    }
                )
                documents.append(doc)
                seq_num += 1
        
        return documents

# 사용 예시
file_path = "/data/ephemeral/home/upstage-ai-final-ir2/HM/data/documents.jsonl"
loader = JSONLLoader(file_path)
documents = loader.load()

print(f"문서의 수: {len(documents)}")

문서의 수: 4272


# 2. 문서 분할(Split Documents)
문서를 chunk로 분할(split)하는 것은 RAG(Retrieval-Augmented Generation) 시스템에서 매우 중요한 단계입니다. 이 과정의 주요 이유와 이점은 다음과 같습니다:

검색 정확도 향상:

큰 문서를 작은 chunk로 나누면 질문과 더 관련성 높은 부분을 정확하게 검색할 수 있습니다.
전체 문서보다 특정 chunk가 질문에 더 적합할 가능성이 높습니다.


컨텍스트 관리:

LLM(Large Language Model)은 입력 토큰 수에 제한이 있습니다. 작은 chunk를 사용하면 이 제한 내에서 더 많은 관련 정보를 포함할 수 있습니다.


검색 효율성:

벡터 데이터베이스에서 작은 chunk를 검색하는 것이 큰 문서 전체를 검색하는 것보다 빠르고 효율적입니다.


메모리 효율성:

작은 chunk는 메모리 사용을 줄이고, 대규모 문서 컬렉션을 효율적으로 처리할 수 있게 합니다.


세분화된 정보 접근:

문서의 특정 부분에 집중할 수 있어, 더 정확하고 관련성 높은 응답을 생성할 수 있습니다.


중복 제거와 정보 필터링:

chunk 생성 과정에서 중복된 정보를 제거하거나 불필요한 부분을 필터링할 수 있습니다.


다양한 소스 통합:

여러 문서의 chunk를 효과적으로 조합하여 종합적인 정보를 제공할 수 있습니다.


컨텍스트 윈도우 최적화:

chunk 크기를 조절하여 LLM에 제공되는 컨텍스트의 양을 최적화할 수 있습니다.


임베딩 품질 향상:

작은 chunk는 더 집중된 의미를 가질 수 있어, 더 정확한 임베딩을 생성할 수 있습니다.


동적 컨텍스트 구성:

질문에 따라 가장 관련성 높은 chunk들을 동적으로 선택하여 컨텍스트를 구성할 수 있습니다.



문서를 chunk로 분할하는 것은 RAG 시스템의 성능, 효율성, 정확성을 크게 향상시킬 수 있는 중요한 전처리 단계입니다. 하지만 적절한 chunk 크기와 분할 방법을 선택하는 것이 중요하며, 이는 문서의 특성과 사용 사례에 따라 달라질 수 있습니다.

In [27]:
# RecursiveCharacterTextSplitter 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

# 문서 분할
split_documents = text_splitter.split_documents(documents)

print(f"문서의 수: {len(documents)}")
print(f"분할된 문서의 수: {len(split_documents)}")

문서의 수: 4272
분할된 문서의 수: 4280


In [60]:
# 문장 단위 TextSplitter
class SentenceSplitter(TextSplitter):
    def split_text(self, text: str) -> list[str]:
        # 문장 끝 패턴: .!?로 끝나고 공백이 따라오는 경우
        # 줄바꿈 문자도 문장의 끝으로 간주
        return [sentence.strip() for sentence in re.split(r'(?<=[.!?])\s+|\n', text) if sentence.strip()]

# SentenceSplitter 초기화
sentence_splitter = SentenceSplitter()

# 문서 분할
sentence_split_documents = sentence_splitter.split_documents(documents)

print(f"원본 문서의 수: {len(documents)}")
print(f"분할된 문서의 수: {len(split_documents)}")

원본 문서의 수: 4272
분할된 문서의 수: 27964


In [50]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [57]:
# SemanticChunker
semantic_text_splitter = SemanticChunker(
    OpenAIEmbeddings(), add_start_index=True)

# documents를 split
semantic_split_documents = semantic_text_splitter.split_documents(documents)

print(f"원본 문서의 수: {len(documents)}")
print(f"분할된 문서의 수: {len(semantic_split_documents)}")

원본 문서의 수: 4272
분할된 문서의 수: 27964


In [63]:
print(f"원본 문서의 수: {len(documents)}")
print(f"분할된 문서의 수: {len(semantic_split_documents)}")

원본 문서의 수: 4272
분할된 문서의 수: 8540


In [58]:
# split_documents 저장
with open('semantic_split_documents.pkl', 'wb') as f:
    pickle.dump(semantic_split_documents, f)

In [59]:
# split_documents 로드
with open('semantic_split_documents.pkl', 'rb') as f:
    loaded_split_documents = pickle.load(f)

print(f"로드된 분할 문서의 수: {len(loaded_split_documents)}")

로드된 분할 문서의 수: 8540


# 3. 임베딩(Embedding) 및 벡터저장소 생성(Create Vectorstore)

In [62]:
# semantic_split_documents로 벡터저장소 생성 
vectorstore = FAISS.from_documents(documents=semantic_split_documents, embedding=OpenAIEmbeddings())

#>> FAISS.from_documents() 
#>> semantic_split_documents의 내용을 OpenAI 임베딩 모델을 통해 고차원 벡터로 변환 
#>> FAISS 인덱스 생성 
#>> 위에서 생성된 문서의 벡터를 FAISS 인덱스에 추가 

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [ ]:
# 유사도 검색 테스트 
query = "금성에서 달이 어떻게 보일까?"
similar_docs = vectorstore.similarity_search(query, k=3)  # 상위 3개 유사 문서 검색
#>> 유클리디안 거리 

for doc in similar_docs:
    print(f"내용: {doc.page_content[:100]}...")
    print(f"메타데이터: {doc.metadata}")
    print("---")

# 4. 벡터스토어 